In [1]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
from glob import glob, iglob
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np; print('numpy', np.__version__)
import pandas as pd; print('pandas', pd.__version__)
import cv2; print('opencv2', cv2.__version__)
import sklearn; print('sklearn', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import tensorflow.keras as keras; print('keras', keras.__version__)

import settings
import helper

Python 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]] on linux
numpy 1.16.4
pandas 0.24.2
opencv2 4.1.0
sklearn 0.21.2
tensorflow 1.14.0
keras 2.2.4-tf


# 1. Load Meta File and Annotation

In [2]:
df_meta_train = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TRAIN_META_FILE, index_col=['seriesuid'])
df_meta_train.index = df_meta_train.index.astype('str')
df_meta_test = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TEST_META_FILE, index_col=['seriesuid'])
df_meta_test.index = df_meta_test.index.astype('str')

print('meta_train:', df_meta_train.shape, '\n', 'meta_test:', df_meta_test.shape)

meta_train: (1470, 11) 
 meta_test: (222, 11)


In [3]:
df_annotation = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_ANNOTATION_FILE, index_col=['seriesuid'])
df_annotation.index = df_annotation.index.astype('str')
df_annotation['label'] = df_annotation['label'].astype('int')
print('annotation:', df_annotation.shape, 'distinct lung:', len(set(df_annotation.index)))

annotation: (12218, 16) distinct lung: 1436


In [4]:
df_annotation.sample(10)

,width,height,slice,vcoordX,vcoordY,vcoordZ,diameterX,diameterY,diameterZ,originX,originY,originZ,spacingX,spacingY,spacingZ,label
seriesuid,,,,,,,,,,,,,,,,
496305,298,298,295,251.437500,137.941406,110.0,6.402344,30.847656,15.0,-169.708984,-308.708984,-307.299988,0.582031,0.582031,5.0,5
525815,360,360,330,119.531250,140.273438,185.0,47.109375,54.843750,5.0,-180.000000,-180.000000,-737.700012,0.703125,0.703125,5.0,5
639356,364,364,350,127.257813,83.179688,140.0,7.820312,12.085938,5.0,-193.991562,-181.644531,-849.779785,0.710938,0.710938,5.0,5
689163,387,387,315,187.453036,123.960879,220.0,9.826167,9.826167,5.0,-194.800003,-193.500000,-890.500000,0.755859,0.755859,5.0,1
369542,350,350,306,186.279297,176.025391,117.0,21.875000,16.406250,1.0,-158.899994,-50.400002,-394.850006,0.683594,0.683594,1.0,31
689227,350,350,355,217.382812,160.644531,135.0,7.519531,6.152344,5.0,-185.149994,-4.000000,-570.840027,0.683594,0.683594,5.0,31
647525,350,350,310,142.405506,193.289008,235.0,27.320001,11.611000,5.0,-174.658005,-174.658203,-845.500000,0.683000,0.683000,5.0,5
524293,303,303,360,111.553711,215.414062,305.0,7.101562,8.876953,5.0,-147.204102,-292.204102,-27.400000,0.591797,0.591797,5.0,1
689092,308,308,235,215.960938,107.980469,72.0,19.851562,31.281250,10.0,-159.699219,-293.699219,-408.600006,0.601562,0.601562,5.0,5


# 2. Load Center Coords by Candidates

In [5]:
lungs = list(set(df_meta_test.index))
print('distinct lungs in testset:', len(lungs))

distinct lungs in testset: 222


In [6]:
list_tasks = []
size = settings.CUBE_SIZE
step = 32
start = [ size//2, size//2, size//2 ] # z,y,x

for uid in tqdm(lungs):
    meta = df_meta_test.loc[uid] # slice
    
    for z in range(start[0], int(meta.slice), step):
        for y in range(start[1], int(meta.height), step):
            for x in range(start[2], int(meta.width), step):
                task = {}
                task['seriesuid'] = uid
                task['vcoordX'] = x
                task['vcoordY'] = y
                task['vcoordZ'] = z
                
                # print(task)
                list_tasks.append(task)

100%|██████████| 222/222 [00:00<00:00, 363.27it/s]


In [7]:
df_tasks = pd.DataFrame(list_tasks, columns=['seriesuid','vcoordX','vcoordY','vcoordZ'])
df_tasks = df_tasks.set_index('seriesuid')
df_tasks.index = df_tasks.index.astype('str')

print('total:', df_tasks.shape, 'lung:', len(set(df_tasks.index)))

if not os.path.exists(settings.SUBMISSION_DIR + 'candidates'):
    os.mkdir(settings.SUBMISSION_DIR + 'candidates')
    
df_tasks.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/tasks_cubes.csv', encoding='utf-8')


total: (208898, 3) lung: 222


# 3. Predict Results by tasks_cubes.csv

In [8]:
# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()

# gpu allow_growth
config.gpu_options.allow_growth = True
keras.backend.set_session(tf.Session(config=config))

In [9]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):
    SMOOTH = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + SMOOTH) / (K.sum(y_true_f) + K.sum(y_pred_f) + SMOOTH)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

label = '32'
output_dir = './output/unet3d/1564368659/' # 32
model = load_model(output_dir + "model-best.hd5", 
                   custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})

W0729 13:04:06.744276 139769041725248 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0729 13:04:06.745175 139769041725248 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0729 13:04:06.751257 139769041725248 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprec

In [10]:
batch_size = 64
threshold_probability = 0.8
wtype = 'medi'
plot = False

In [11]:
def plot_cube(cube, rows=8, cols=8):
    num_z, h, w = cube.shape # z,y,x
    assert rows * cols == num_z
    
    img = np.zeros((rows * h, cols * w), dtype=np.uint8)

    for row in range(rows):
        for col in range(cols):
            y = row * h
            x = col * w
            img[y:(y+h), x:(x+w)] = cube[row * cols + col,:,:]
    
    fig, axs = plt.subplots(1, 1, figsize=(32, 32), sharex=True, sharey=True)
    axs.imshow(img, cmap='gray')
    plt.show()
    

In [12]:
list_results = []
print(wtype, 'window')
for uid in set(df_tasks.index):
    tasks = df_tasks.loc[[uid]] # DataFrame
    meta = df_meta_test.loc[uid] # slice
    
    print('lung:', uid, 'tasks:', len(tasks), 'meta:', meta.slice, meta.height, meta.width)
#     print(meta)
                 

    lung_l, mask_l = helper.load_lung_array(uid, int(meta.width), int(meta.height), int(meta.slice), 
                                            mode='test', wtype=wtype)

    lung = (lung_l*(mask_l>0)).astype('float') / 255

    count = 0
    for idx, item in tasks.iterrows():
        if count % (batch_size*100) == 0:
            X_item = []
            X_test = []

        # patch without mask
        cube = helper.get_cube_from_lung_array(lung, item.vcoordX, item.vcoordY, item.vcoordZ, 
                                               block_size=settings.CUBE_SIZE)

        if np.sum(cube) > 0:
            X_item.append(item)
            X_test.append(cube)
#         else:
#             print('nothing in coord:', '\n', item)

        if (count % (batch_size*100) == (batch_size*100 - 1) or count == len(tasks) - 1) and len(X_test) > 0:
            assert len(X_item) == len(X_test) or count == len(tasks) - 1

            # Make predictions
            X_test = np.expand_dims(np.asarray(X_test), axis=-1)
            print(X_test.shape)
            predictions_test = model.predict(X_test, batch_size=batch_size, verbose=1)

            for i in range(len(X_item)):
                itm = X_item[i]
                pred = predictions_test[i] > threshold_probability
                if np.sum(pred) > 10:
                    vcoords, diameters, bboxes = helper.get_regions_detail(pred[:,:,:,0], itm)
                    if len(bboxes) > 0:
                        for i, box in enumerate(bboxes):
                            result = {}
                            result['seriesuid'] = uid
                            result['vcoordX'] = vcoords[i][2]
                            result['vcoordY'] = vcoords[i][1]
                            result['vcoordZ'] = vcoords[i][0]
                            result['diametersX'] = diameters[i][2]
                            result['diametersY'] = diameters[i][1]
                            result['diametersZ'] = diameters[i][0]
                            if plot:
                                plot_cube((pred*255).astype(np.uint8))

                            list_results.append(result)
                        
        count += 1
   
    # save the csv by per lung
    df_results = pd.DataFrame(list_results, columns=['seriesuid','vcoordX','vcoordY','vcoordZ','diametersX','diametersY','diametersZ'])
    df_results = df_results.set_index('seriesuid')
    df_results.index = df_results.index.astype('str')

    print('total:', df_results.shape, 'lung:', len(set(df_results.index)))

    df_results.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype + '_' + label + '.csv', encoding='utf-8')

    print('-'*100)
    

medi window
lung: 689056 tasks: 1210 meta: 325.0 374.0 374.0
(1176, 64, 64, 64, 1)
1176/1176 [==============================] - 23s 19ms/sample
total: (106, 6) lung: 1
----------------------------------------------------------------------------------------------------
lung: 680488 tasks: 800 meta: 270.0 329.0 329.0
(797, 64, 64, 64, 1)
797/797 [==============================] - 12s 15ms/sample
total: (218, 6) lung: 2
----------------------------------------------------------------------------------------------------
lung: 694572 tasks: 729 meta: 320.0 305.0 305.0
(704, 64, 64, 64, 1)
704/704 [==============================] - 10s 14ms/sample
total: (305, 6) lung: 3
----------------------------------------------------------------------------------------------------
lung: 637259 tasks: 900 meta: 320.0 350.0 350.0
(884, 64, 64, 64, 1)
884/884 [==============================] - 14s 15ms/sample
total: (414, 6) lung: 4
-------------------------------------------------------------------------

(687, 64, 64, 64, 1)
687/687 [==============================] - 15s 21ms/sample
total: (3366, 6) lung: 33
----------------------------------------------------------------------------------------------------
lung: 686233 tasks: 900 meta: 295.0 350.0 350.0
(847, 64, 64, 64, 1)
847/847 [==============================] - 14s 17ms/sample
total: (3461, 6) lung: 34
----------------------------------------------------------------------------------------------------
lung: 647851 tasks: 900 meta: 315.0 350.0 350.0
(850, 64, 64, 64, 1)
850/850 [==============================] - 18s 21ms/sample
total: (3542, 6) lung: 35
----------------------------------------------------------------------------------------------------
lung: 658163 tasks: 800 meta: 265.0 350.0 350.0
(703, 64, 64, 64, 1)
703/703 [==============================] - 15s 22ms/sample
total: (3626, 6) lung: 36
----------------------------------------------------------------------------------------------------
lung: 639652 tasks: 900 meta

749/749 [==============================] - 16s 21ms/sample
total: (6500, 6) lung: 65
----------------------------------------------------------------------------------------------------
lung: 526336 tasks: 640 meta: 330.0 285.0 285.0
(602, 64, 64, 64, 1)
602/602 [==============================] - 10s 16ms/sample
total: (6594, 6) lung: 66
----------------------------------------------------------------------------------------------------
lung: 680322 tasks: 1008 meta: 250.0 394.0 394.0
(926, 64, 64, 64, 1)
926/926 [==============================] - 16s 17ms/sample
total: (6688, 6) lung: 67
----------------------------------------------------------------------------------------------------
lung: 689297 tasks: 1000 meta: 325.0 350.0 350.0
(870, 64, 64, 64, 1)
870/870 [==============================] - 15s 17ms/sample
total: (6796, 6) lung: 68
----------------------------------------------------------------------------------------------------
lung: 655422 tasks: 900 meta: 315.0 327.0 327.0

1253/1253 [==============================] - 21s 17ms/sample
total: (9558, 6) lung: 97
----------------------------------------------------------------------------------------------------
lung: 633846 tasks: 1296 meta: 305.0 403.0 403.0
(1110, 64, 64, 64, 1)
1110/1110 [==============================] - 18s 16ms/sample
total: (9665, 6) lung: 98
----------------------------------------------------------------------------------------------------
lung: 672337 tasks: 700 meta: 255.0 350.0 350.0
(678, 64, 64, 64, 1)
678/678 [==============================] - 11s 16ms/sample
total: (9750, 6) lung: 99
----------------------------------------------------------------------------------------------------
lung: 660058 tasks: 1210 meta: 345.0 369.0 369.0
(1186, 64, 64, 64, 1)
1186/1186 [==============================] - 22s 18ms/sample
total: (9872, 6) lung: 100
----------------------------------------------------------------------------------------------------
lung: 633135 tasks: 700 meta: 255.0 34

(719, 64, 64, 64, 1)
719/719 [==============================] - 11s 15ms/sample
total: (12650, 6) lung: 129
----------------------------------------------------------------------------------------------------
lung: 652031 tasks: 900 meta: 290.0 350.0 350.0
(843, 64, 64, 64, 1)
843/843 [==============================] - 14s 17ms/sample
total: (12751, 6) lung: 130
----------------------------------------------------------------------------------------------------
lung: 656481 tasks: 900 meta: 296.0 330.0 330.0
(879, 64, 64, 64, 1)
448/879 [==============>...............] - ETA: 9s 

KeyboardInterrupt: 

In [13]:
df_results = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype + '_' + label +'.csv', 
                            index_col=['seriesuid'])
df_results.index = df_results.index.astype('str')

print('results:', df_results.shape, 'distinct lung:', len(set(df_results.index)))



results: (12751, 6) distinct lung: 130


In [14]:
filepath_31 = settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype +'_31.csv'
filepath_32 = settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype +'_32.csv'
if os.path.exists(filepath_31) and os.path.exists(filepath_32):
    df_medi_31 = pd.read_csv(filepath_or_buffer=filepath_31, index_col=['seriesuid'])
    df_medi_31.index = df_medi_31.index.astype('str')
    print('medi window 31:', df_medi_31.shape)
    
    df_medi_32 = pd.read_csv(filepath_or_buffer=filepath_32, index_col=['seriesuid'])
    df_medi_32.index = df_medi_32.index.astype('str')
    print('medi window 32:', df_medi_32.shape)
    
    df_medi = pd.concat([df_medi_31, df_medi_32])
    print('medi windows merged:', df_medi.shape)
    df_medi.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype +'.csv', encoding='utf-8')

medi window 31: (26124, 6)
medi window 32: (12751, 6)
medi windows merged: (38875, 6)
